In [2]:
!pip install torchmetrics==0.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import time
import os

import cv2
from torchmetrics import F1Score
from torchmetrics.functional import precision_recall
from torchmetrics.classification import BinaryAUROC
import pickle

In [4]:
path = '/kaggle/input/traffic-sign-datasets/'
path_dataset1 = path+'Indian Traffic Signs Prediction(85 classes)/Indian Traffic Signs Prediction(85 classes)/'
path_dataset2 = path+'Persian Traffic Sign Dataset (PTSD)/Persian Traffic Sign Dataset (PTSD)/'
path_dataset3 = path+'Traffic Signs (GTSRB plus 162 custom classes)/Traffic Signs (GTSRB plus 162 custom classes)/Data_images/'

savepath='/kaggle/working/'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path='/content/drive/My Drive/6721 Dataset/6721 Project dataset/'
savepath = '/content/drive/My Drive/6721 Dataset/'
print(os.listdir(path))

['Indian Traffic Signs Prediction(85 classes)', 'Persian Traffic Sign Dataset (PTSD)', 'Traffic Signs (GTSRB plus 162 custom classes)']


In [8]:
path_dataset1 = path+"Indian Traffic Signs Prediction(85 classes)/"
path_dataset2 = path+"Persian Traffic Sign Dataset (PTSD)/"
path_dataset3 = path+"Traffic Signs (GTSRB plus 162 custom classes)/Data_images/"

**Data Loaders**

---



In [11]:
def load_data(path_train, val_split, path_test, batch_size, input_size):
  
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    transform_train= transforms.Compose([transforms.Resize(input_size),
                                         transforms.ColorJitter(brightness=(0.5,1.2), contrast=0, saturation=0, hue=0),
                                         transforms.RandomHorizontalFlip(),
#                                          transforms.RandomVerticalFlip(),
                                          transforms.RandomRotation(45),
                                        transforms.RandomAdjustSharpness(0.8),
                                          transforms.ToTensor(),
                                          normalize
                                          ])

    transform_test = transforms.Compose([transforms.Resize(input_size),
                                         transforms.ToTensor(), 
                                         normalize])

    data_train = datasets.ImageFolder(root=path_train, transform=transform_train)
    data_test = datasets.ImageFolder(root=path_test, transform=transform_test)
    
    mappings = data_train.class_to_idx

    val_size = int(len(data_train)*val_split)
    train_size = len(data_train) - val_size

    train_dataset, val_dataset = td.random_split(data_train, [train_size, val_size])
    
    data_loader_train = td.DataLoader(train_dataset,
                                      batch_size=batch_size,
                                      shuffle=True,
                                      drop_last=False,
                                      num_workers=0,       
                                      pin_memory=True)
     
    data_loader_val = td.DataLoader(val_dataset,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    drop_last=False,
                                    num_workers=0) 
      
    data_loader_test = td.DataLoader(data_test,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   drop_last=False,
                                   num_workers=0)
    
    return data_loader_train, data_loader_test, data_loader_val, mappings

Compute Class Weights

In [12]:
def get_image_size_dist(path):
    t1 = time.time()

    folders = os.listdir(path)
    total = 0

    class_samples = {}

    for folder in folders:
        samples = os.listdir(path+folder)

        class_samples[folder]=len(samples)

        print("Class: {} has samples: {} ".format(folder,len(samples)))
        total+=len(samples)

    print("\nTotal number of samples: ",total)

    print("\nTime taken: {}".format(time.time()-t1))

    return class_samples, total

In [13]:
def get_class_weights(path_train, mappings):

    class_samples, total = get_image_size_dist(path_train)

    class_weights = [0]*len(mappings.keys())

    for key in mappings.keys():
        class_weights[mappings[key]] = 1 - (class_samples[key]/ total)
      
    return class_weights

Training Setup

---



In [14]:
learning_rate = 0.001
num_epochs = 10

def get_criterion(class_weights):

    criterion = nn.CrossEntropyLoss(weight = class_weights)

    return criterion

In [15]:
def train_model(model, num_epochs, train_loader, criterion, optimizer, savepath, savepath_results, device):

      total_steps = len(train_loader)

      t1 = time.time()

      train_loss=[]
      train_accuracy=[]
      val_accuracy=[]

      for epoch in range(num_epochs):
          for i, data in enumerate(train_loader):
              
              images, labels = data[0].to(device), data[1].to(device)
              
              model.train()

              # Forward pass
              outputs = model(images)
              loss = criterion(outputs, labels)
              
              # Backprop and optimisation
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()
              
              # Train accuracy
              total = labels.size(0)
              _, predicted = torch.max(outputs.data, 1)
              correct = (predicted == labels).sum().item()
              
              if (i + 1) % 10 == 0:
                  model.eval() 
                  with torch.no_grad(): 
                      correctv = 0
                      totalv = 0
                      for datav in val_loader:
                          imagesv, labelsv = datav[0].to(device), datav[1].to(device)
                          outputsv = model(imagesv)
                          _, predictedv = torch.max(outputsv.data, 1)
                          totalv += labelsv.size(0)
                          correctv += (predictedv == labelsv).sum().item()
                          
                      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%, Validation Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                          (correct / total) * 100,
                          (correctv / totalv) * 100))
                      
                      train_loss.append(loss.item())
                      train_accuracy.append((correct / total) * 100)
                      val_accuracy.append((correctv / totalv) * 100)
                      
                  
      print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

      print("/n/n Saving model at: ",savepath)
      torch.save(model.state_dict(), savepath)

      results = { "training_loss": train_loss, "accuracy": train_accuracy, "val_accuracy": val_accuracy }

      with open(savepath_results, "wb") as fp:
          pickle.dump(results, fp)

      return model, device

In [16]:
def evaluate_model(model, test_loader, device, nclasses):
   
    model.eval() 

    Y=[]
    y=[]

    with torch.no_grad(): 
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            Y.extend(labels)
            y.extend(predicted)

        acc = (correct / total) * 100
        f1 = F1Score(num_classes=nclasses)
        f1score = f1(torch.IntTensor(y),torch.IntTensor(Y))

        pr = precision_recall(torch.IntTensor(y),torch.IntTensor(Y), average='macro', num_classes=nclasses)
        auroc = BinaryAUROC(thresholds=None, num_classes=nclasses)
        auc= auroc(torch.IntTensor(y),torch.IntTensor(Y))
        
        print('Model Evaluation Results on {} test samples'.format(total))
        print('Test Accuracy: ', acc)
        print('Test F1 Score: ',f1score)
        print('Test Precision & Recall:', pr)
        print('Test AUC score:',auc)
        

Architecture 1: AlexNet

On dataset 1:

In [17]:
nclasses=15

In [18]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)

Downloading: "https://github.com/pytorch/vision/zipball/v0.6.0" to /root/.cache/torch/hub/v0.6.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [19]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [20]:
AlexNet_model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [21]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [22]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = AlexNet_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: SPEED_LIMIT_80 has samples: 192 
Class: SPEED_LIMIT_60 has samples: 190 
Class: NO_STOPPING_OR_STANDING has samples: 242 
Class: COMPULSARY_KEEP_RIGHT has samples: 223 
Class: GAP_IN_MEDIAN has samples: 180 
Class: CROSS_ROAD has samples: 140 
Class: PEDESTRIAN_CROSSING has samples: 121 
Class: SPEED_LIMIT_50 has samples: 200 
Class: HORN_PROHIBITED has samples: 160 
Class: SPEED_LIMIT_40 has samples: 170 
Class: NO_ENTRY has samples: 174 
Class: HUMP_OR_ROUGH_ROAD has samples: 101 
Class: SPEED_LIMIT_30 has samples: 238 
Class: LEFT_TURN_PROHIBITED has samples: 126 
Class: SPEED_LIMIT_70 has samples: 160 

Total number of samples:  2617

Time taken: 0.008558988571166992


In [ ]:
model_savepath = savepath+"alexnetmodel_d1.pt"
results_savepath = savepath+"alexnetmodel_d1_results.pkb"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/66], Loss: 2.7295, Accuracy: 6.25%, Validation Accuracy: 7.07%
Epoch [1/10], Step [20/66], Loss: 2.6838, Accuracy: 9.38%, Validation Accuracy: 7.07%
Epoch [1/10], Step [30/66], Loss: 2.6286, Accuracy: 12.50%, Validation Accuracy: 7.07%
Epoch [1/10], Step [40/66], Loss: 2.6480, Accuracy: 15.62%, Validation Accuracy: 7.07%
Epoch [1/10], Step [50/66], Loss: 2.6554, Accuracy: 9.38%, Validation Accuracy: 7.07%
Epoch [1/10], Step [60/66], Loss: 2.6864, Accuracy: 3.12%, Validation Accuracy: 8.03%
Epoch [2/10], Step [10/66], Loss: 2.6617, Accuracy: 9.38%, Validation Accuracy: 17.21%
Epoch [2/10], Step [20/66], Loss: 2.4279, Accuracy: 18.75%, Validation Accuracy: 21.99%
Epoch [2/10], Step [30/66], Loss: 2.1817, Accuracy: 28.12%, Validation Accuracy: 14.53%
Epoch [2/10], Step [40/66], Loss: 2.3154, Accuracy: 28.12%, Validation Accuracy: 16.44%
Epoch [2/10], Step [50/66], Loss: 2.4308, Accuracy: 9.38%, Validation Accuracy: 21.61%
Epoch [2/10], Step [60/66], Loss: 2.2330, Ac

In [24]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Model Evaluation Results on 710 test samples
Test Accuracy:  55.21126760563381
Test F1 Score:  tensor(0.5521)
Test Precision & Recall: (tensor(0.6320), tensor(0.5853))
Test AUC score: tensor(0.0738)


On dataset 2:

In [25]:
nclasses=12

In [26]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [27]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [28]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [31]:
path_train = path_dataset2+"train/"
path_test = path_dataset2+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = AlexNet_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: Pedestrain has samples: 1014 
Class: NO Stopping has samples: 1075 
Class: Right Turn Prohibited has samples: 436 
Class: Danger has samples: 399 
Class: Compulsory Keep BothSide has samples: 573 
Class: Maximum Speed 30 has samples: 477 
Class: Right Margin has samples: 506 
Class: Cycle crossing has samples: 662 
Class: Roundabouts has samples: 497 
Class: One way Traffic has samples: 684 
Class: Compulsory Keep Right has samples: 703 
Class: No Entry has samples: 930 

Total number of samples:  7956

Time taken: 0.008566141128540039


In [ ]:
model_savepath = savepath+"alexnetmodel_d2.pt"
results_savepath = savepath+"alexnetmodel_d2_results.pkb"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/199], Loss: 2.4676, Accuracy: 12.50%, Validation Accuracy: 14.20%
Epoch [1/10], Step [20/199], Loss: 2.4558, Accuracy: 12.50%, Validation Accuracy: 14.20%
Epoch [1/10], Step [30/199], Loss: 2.3851, Accuracy: 31.25%, Validation Accuracy: 14.20%
Epoch [1/10], Step [40/199], Loss: 2.3879, Accuracy: 9.38%, Validation Accuracy: 14.20%
Epoch [1/10], Step [50/199], Loss: 2.3227, Accuracy: 9.38%, Validation Accuracy: 12.70%
Epoch [1/10], Step [60/199], Loss: 2.1721, Accuracy: 15.62%, Validation Accuracy: 14.20%
Epoch [1/10], Step [70/199], Loss: 2.3524, Accuracy: 28.12%, Validation Accuracy: 16.91%
Epoch [1/10], Step [80/199], Loss: 2.2265, Accuracy: 28.12%, Validation Accuracy: 17.85%
Epoch [1/10], Step [90/199], Loss: 2.3676, Accuracy: 18.75%, Validation Accuracy: 14.20%
Epoch [1/10], Step [100/199], Loss: 2.0491, Accuracy: 18.75%, Validation Accuracy: 22.25%
Epoch [1/10], Step [110/199], Loss: 2.0274, Accuracy: 28.12%, Validation Accuracy: 23.44%
Epoch [1/10], Step [1

In [33]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Model Evaluation Results on 1228 test samples
Test Accuracy:  91.77524429967427
Test F1 Score:  tensor(0.9178)
Test Precision & Recall: (tensor(0.9079), tensor(0.9272))
Test AUC score: tensor(0.0755)


On dataset 3:

In [34]:
nclasses=8

In [35]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [36]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [37]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [38]:
path_train = path_dataset3+"Train/"
path_test = path_dataset3+"Test/"

val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = AlexNet_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: 7 has samples: 1433 
Class: 2 has samples: 2243 
Class: 5 has samples: 1853 
Class: 8 has samples: 1403 
Class: 3 has samples: 1403 
Class: 1 has samples: 2213 
Class: 4 has samples: 1973 
Class: 9 has samples: 1463 

Total number of samples:  13984

Time taken: 0.008931398391723633


In [ ]:
model_savepath = savepath+"alexnetmodel_d3.pt"
results_savepath = savepath+"alexnetmodel_d3_results.pkb"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/350], Loss: 2.0695, Accuracy: 3.12%, Validation Accuracy: 15.95%
Epoch [1/10], Step [20/350], Loss: 2.0585, Accuracy: 12.50%, Validation Accuracy: 14.70%
Epoch [1/10], Step [30/350], Loss: 2.0732, Accuracy: 9.38%, Validation Accuracy: 18.99%
Epoch [1/10], Step [40/350], Loss: 2.0003, Accuracy: 25.00%, Validation Accuracy: 19.46%
Epoch [1/10], Step [50/350], Loss: 2.0627, Accuracy: 15.62%, Validation Accuracy: 15.09%
Epoch [1/10], Step [60/350], Loss: 2.0262, Accuracy: 18.75%, Validation Accuracy: 16.77%
Epoch [1/10], Step [70/350], Loss: 1.9888, Accuracy: 28.12%, Validation Accuracy: 19.46%
Epoch [1/10], Step [80/350], Loss: 2.0498, Accuracy: 25.00%, Validation Accuracy: 20.39%
Epoch [1/10], Step [90/350], Loss: 1.9850, Accuracy: 15.62%, Validation Accuracy: 17.06%
Epoch [1/10], Step [100/350], Loss: 1.9533, Accuracy: 25.00%, Validation Accuracy: 18.56%
Epoch [1/10], Step [110/350], Loss: 1.9968, Accuracy: 15.62%, Validation Accuracy: 18.96%
Epoch [1/10], Step [1

In [40]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Model Evaluation Results on 4590 test samples
Test Accuracy:  80.13071895424837
Test F1 Score:  tensor(0.8013)
Test Precision & Recall: (tensor(0.7939), tensor(0.7949))
Test AUC score: tensor(0.2800)


Architecture 2: VGG-11

on dataset 1:

In [41]:
nclasses=15

In [42]:
VGG11_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=False)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


In [43]:
prev_out = VGG11_model.classifier[3].out_features
VGG11_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [44]:
VGG11_model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [45]:
optimizer = torch.optim.Adam(VGG11_model.parameters(), lr=learning_rate) 

In [46]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = VGG11_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: SPEED_LIMIT_80 has samples: 192 
Class: SPEED_LIMIT_60 has samples: 190 
Class: NO_STOPPING_OR_STANDING has samples: 242 
Class: COMPULSARY_KEEP_RIGHT has samples: 223 
Class: GAP_IN_MEDIAN has samples: 180 
Class: CROSS_ROAD has samples: 140 
Class: PEDESTRIAN_CROSSING has samples: 121 
Class: SPEED_LIMIT_50 has samples: 200 
Class: HORN_PROHIBITED has samples: 160 
Class: SPEED_LIMIT_40 has samples: 170 
Class: NO_ENTRY has samples: 174 
Class: HUMP_OR_ROUGH_ROAD has samples: 101 
Class: SPEED_LIMIT_30 has samples: 238 
Class: LEFT_TURN_PROHIBITED has samples: 126 
Class: SPEED_LIMIT_70 has samples: 160 

Total number of samples:  2617

Time taken: 0.012342691421508789


In [ ]:
model_savepath = savepath+"vgg11model_d1.pt"
results_savepath = savepath+"vgg11model_d1_results.pkb"

trained_VGG11_model, device = train_model(VGG11_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/66], Loss: 2.7052, Accuracy: 3.12%, Validation Accuracy: 5.74%
Epoch [1/10], Step [20/66], Loss: 2.7081, Accuracy: 9.38%, Validation Accuracy: 9.18%
Epoch [1/10], Step [30/66], Loss: 2.6994, Accuracy: 9.38%, Validation Accuracy: 9.18%
Epoch [1/10], Step [40/66], Loss: 2.7006, Accuracy: 9.38%, Validation Accuracy: 9.18%
Epoch [1/10], Step [50/66], Loss: 2.6794, Accuracy: 9.38%, Validation Accuracy: 9.18%
Epoch [1/10], Step [60/66], Loss: 2.6460, Accuracy: 9.38%, Validation Accuracy: 8.03%
Epoch [2/10], Step [10/66], Loss: 2.7304, Accuracy: 6.25%, Validation Accuracy: 8.03%
Epoch [2/10], Step [20/66], Loss: 2.6969, Accuracy: 6.25%, Validation Accuracy: 6.88%
Epoch [2/10], Step [30/66], Loss: 2.7227, Accuracy: 9.38%, Validation Accuracy: 6.88%
Epoch [2/10], Step [40/66], Loss: 2.7500, Accuracy: 9.38%, Validation Accuracy: 8.03%
Epoch [2/10], Step [50/66], Loss: 2.6857, Accuracy: 3.12%, Validation Accuracy: 8.03%
Epoch [2/10], Step [60/66], Loss: 2.6921, Accuracy: 9.

In [48]:
evaluate_model(trained_VGG11_model, test_loader, device, nclasses)

Model Evaluation Results on 710 test samples
Test Accuracy:  11.408450704225352
Test F1 Score:  tensor(0.1141)
Test Precision & Recall: (tensor(0.0076), tensor(0.0667))
Test AUC score: tensor(0.5000)


on dataset 2:

In [49]:
nclasses=12

In [50]:
VGG11_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [51]:
prev_out = VGG11_model.classifier[3].out_features
VGG11_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [52]:
optimizer = torch.optim.Adam(VGG11_model.parameters(), lr=learning_rate) 

In [53]:
path_train = path_dataset2+"train/"
path_test = path_dataset2+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = VGG11_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: Pedestrain has samples: 1014 
Class: NO Stopping has samples: 1075 
Class: Right Turn Prohibited has samples: 436 
Class: Danger has samples: 399 
Class: Compulsory Keep BothSide has samples: 573 
Class: Maximum Speed 30 has samples: 477 
Class: Right Margin has samples: 506 
Class: Cycle crossing has samples: 662 
Class: Roundabouts has samples: 497 
Class: One way Traffic has samples: 684 
Class: Compulsory Keep Right has samples: 703 
Class: No Entry has samples: 930 

Total number of samples:  7956

Time taken: 0.009800910949707031


In [ ]:
model_savepath = savepath+"vgg11model_d2.pt"
results_savepath = savepath+"vgg11model_d2_results.pkb"

trained_VGG11_model, device = train_model(VGG11_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/199], Loss: 2.4733, Accuracy: 6.25%, Validation Accuracy: 11.50%
Epoch [1/10], Step [20/199], Loss: 2.4771, Accuracy: 12.50%, Validation Accuracy: 11.50%
Epoch [1/10], Step [30/199], Loss: 2.4711, Accuracy: 12.50%, Validation Accuracy: 12.51%
Epoch [1/10], Step [40/199], Loss: 2.4288, Accuracy: 12.50%, Validation Accuracy: 11.50%
Epoch [1/10], Step [50/199], Loss: 2.4787, Accuracy: 15.62%, Validation Accuracy: 12.51%
Epoch [1/10], Step [60/199], Loss: 2.4724, Accuracy: 15.62%, Validation Accuracy: 11.82%
Epoch [1/10], Step [70/199], Loss: 2.4792, Accuracy: 9.38%, Validation Accuracy: 11.50%
Epoch [1/10], Step [80/199], Loss: 2.5084, Accuracy: 9.38%, Validation Accuracy: 11.50%
Epoch [1/10], Step [90/199], Loss: 2.4381, Accuracy: 9.38%, Validation Accuracy: 11.50%
Epoch [1/10], Step [100/199], Loss: 2.4098, Accuracy: 12.50%, Validation Accuracy: 11.50%
Epoch [1/10], Step [110/199], Loss: 2.3562, Accuracy: 21.88%, Validation Accuracy: 13.14%
Epoch [1/10], Step [120

In [55]:
evaluate_model(trained_VGG11_model, test_loader, device, nclasses)

Model Evaluation Results on 1228 test samples
Test Accuracy:  95.11400651465797
Test F1 Score:  tensor(0.9511)
Test Precision & Recall: (tensor(0.9397), tensor(0.9455))
Test AUC score: tensor(0.0644)


on dataset 3:

In [56]:
nclasses=8

In [57]:
VGG11_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [58]:
prev_out = VGG11_model.classifier[3].out_features
VGG11_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [59]:
optimizer = torch.optim.Adam(VGG11_model.parameters(), lr=learning_rate) 

In [60]:
path_train = path_dataset3+"Train/"
path_test = path_dataset3+"Test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = VGG11_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: 1 has samples: 2213 
Class: 5 has samples: 1853 
Class: 7 has samples: 1433 
Class: 2 has samples: 2243 
Class: 3 has samples: 1413 
Class: 4 has samples: 1973 
Class: 9 has samples: 1463 
Class: 8 has samples: 1403 

Total number of samples:  13994

Time taken: 0.15865254402160645


In [ ]:
model_savepath = savepath+"vgg11model_d3.pt"
results_savepath = savepath+"vgg11model_d3_results.pkb"

trained_VGG11_model, device = train_model(VGG11_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/350], Loss: 2.0727, Accuracy: 28.12%, Validation Accuracy: 15.30%
Epoch [1/10], Step [20/350], Loss: 2.0566, Accuracy: 9.38%, Validation Accuracy: 16.48%
Epoch [1/10], Step [30/350], Loss: 2.0608, Accuracy: 12.50%, Validation Accuracy: 13.58%
Epoch [1/10], Step [40/350], Loss: 2.0757, Accuracy: 9.38%, Validation Accuracy: 18.01%
Epoch [1/10], Step [50/350], Loss: 2.0853, Accuracy: 6.25%, Validation Accuracy: 15.30%
Epoch [1/10], Step [60/350], Loss: 2.0770, Accuracy: 25.00%, Validation Accuracy: 16.48%
Epoch [1/10], Step [70/350], Loss: 2.0321, Accuracy: 15.62%, Validation Accuracy: 15.30%
Epoch [1/10], Step [80/350], Loss: 2.0804, Accuracy: 6.25%, Validation Accuracy: 15.30%
Epoch [1/10], Step [90/350], Loss: 2.0162, Accuracy: 28.12%, Validation Accuracy: 15.30%
Epoch [1/10], Step [100/350], Loss: 2.1865, Accuracy: 6.25%, Validation Accuracy: 16.55%
Epoch [1/10], Step [110/350], Loss: 2.0735, Accuracy: 25.00%, Validation Accuracy: 15.30%
Epoch [1/10], Step [120/

In [62]:
evaluate_model(trained_VGG11_model, test_loader, device, nclasses)

Model Evaluation Results on 4590 test samples
Test Accuracy:  91.98257080610021
Test F1 Score:  tensor(0.9198)
Test Precision & Recall: (tensor(0.9253), tensor(0.9173))
Test AUC score: tensor(0.2126)


Architecture 3: ResNet-18

on dataset 1:

In [63]:
nclasses=15

In [64]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [65]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [66]:
ResNet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [67]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [68]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = ResNet_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: SPEED_LIMIT_80 has samples: 192 
Class: SPEED_LIMIT_60 has samples: 190 
Class: NO_STOPPING_OR_STANDING has samples: 242 
Class: COMPULSARY_KEEP_RIGHT has samples: 223 
Class: GAP_IN_MEDIAN has samples: 180 
Class: CROSS_ROAD has samples: 140 
Class: PEDESTRIAN_CROSSING has samples: 121 
Class: SPEED_LIMIT_50 has samples: 200 
Class: HORN_PROHIBITED has samples: 160 
Class: SPEED_LIMIT_40 has samples: 170 
Class: NO_ENTRY has samples: 174 
Class: HUMP_OR_ROUGH_ROAD has samples: 101 
Class: SPEED_LIMIT_30 has samples: 238 
Class: LEFT_TURN_PROHIBITED has samples: 126 
Class: SPEED_LIMIT_70 has samples: 160 

Total number of samples:  2617

Time taken: 0.012465476989746094


In [ ]:
model_savepath = savepath+"resnetmodel_d1.pt"
results_savepath = savepath+"resnetmodel_d1_results.pkb"

trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/66], Loss: 2.5484, Accuracy: 21.88%, Validation Accuracy: 16.06%
Epoch [1/10], Step [20/66], Loss: 2.0677, Accuracy: 25.00%, Validation Accuracy: 23.33%
Epoch [1/10], Step [30/66], Loss: 1.9504, Accuracy: 21.88%, Validation Accuracy: 26.58%
Epoch [1/10], Step [40/66], Loss: 1.2429, Accuracy: 62.50%, Validation Accuracy: 39.39%
Epoch [1/10], Step [50/66], Loss: 1.5976, Accuracy: 40.62%, Validation Accuracy: 36.90%
Epoch [1/10], Step [60/66], Loss: 1.2295, Accuracy: 53.12%, Validation Accuracy: 36.14%
Epoch [2/10], Step [10/66], Loss: 1.3550, Accuracy: 56.25%, Validation Accuracy: 42.45%
Epoch [2/10], Step [20/66], Loss: 1.4624, Accuracy: 59.38%, Validation Accuracy: 37.67%
Epoch [2/10], Step [30/66], Loss: 1.2123, Accuracy: 62.50%, Validation Accuracy: 41.30%
Epoch [2/10], Step [40/66], Loss: 1.8080, Accuracy: 37.50%, Validation Accuracy: 33.27%
Epoch [2/10], Step [50/66], Loss: 1.4519, Accuracy: 56.25%, Validation Accuracy: 32.50%
Epoch [2/10], Step [60/66], Loss

In [70]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)

Model Evaluation Results on 710 test samples
Test Accuracy:  62.25352112676056
Test F1 Score:  tensor(0.6225)
Test Precision & Recall: (tensor(0.6610), tensor(0.6489))
Test AUC score: tensor(0.1070)


on dataset 2:

In [71]:
nclasses=12

In [72]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [73]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [74]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [75]:
path_train = path_dataset2+"train/"
path_test = path_dataset2+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x= ResNet_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: Pedestrain has samples: 1014 
Class: NO Stopping has samples: 1075 
Class: Right Turn Prohibited has samples: 436 
Class: Danger has samples: 399 
Class: Compulsory Keep BothSide has samples: 573 
Class: Maximum Speed 30 has samples: 477 
Class: Right Margin has samples: 506 
Class: Cycle crossing has samples: 662 
Class: Roundabouts has samples: 497 
Class: One way Traffic has samples: 684 
Class: Compulsory Keep Right has samples: 703 
Class: No Entry has samples: 930 

Total number of samples:  7956

Time taken: 0.010478019714355469


In [ ]:
model_savepath = savepath+"resnetmodel_d2.pt"
results_savepath = savepath+"resnetmodel_d2_results.pkb"

trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/199], Loss: 1.8990, Accuracy: 25.00%, Validation Accuracy: 15.21%
Epoch [1/10], Step [20/199], Loss: 2.1102, Accuracy: 31.25%, Validation Accuracy: 33.44%
Epoch [1/10], Step [30/199], Loss: 1.7734, Accuracy: 50.00%, Validation Accuracy: 30.17%
Epoch [1/10], Step [40/199], Loss: 1.8441, Accuracy: 28.12%, Validation Accuracy: 26.84%
Epoch [1/10], Step [50/199], Loss: 1.3490, Accuracy: 50.00%, Validation Accuracy: 45.63%
Epoch [1/10], Step [60/199], Loss: 1.2496, Accuracy: 53.12%, Validation Accuracy: 50.03%
Epoch [1/10], Step [70/199], Loss: 1.5145, Accuracy: 50.00%, Validation Accuracy: 41.48%
Epoch [1/10], Step [80/199], Loss: 1.5542, Accuracy: 65.62%, Validation Accuracy: 52.36%
Epoch [1/10], Step [90/199], Loss: 1.3302, Accuracy: 50.00%, Validation Accuracy: 57.39%
Epoch [1/10], Step [100/199], Loss: 1.8357, Accuracy: 46.88%, Validation Accuracy: 40.92%
Epoch [1/10], Step [110/199], Loss: 1.2714, Accuracy: 46.88%, Validation Accuracy: 49.59%
Epoch [1/10], Step 

In [77]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)

Model Evaluation Results on 1228 test samples
Test Accuracy:  96.41693811074919
Test F1 Score:  tensor(0.9642)
Test Precision & Recall: (tensor(0.9539), tensor(0.9657))
Test AUC score: tensor(0.0858)


on dataset 3:

In [78]:
nclasses=8

In [79]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [80]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [81]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [82]:
path_train = path_dataset3+"Train/"
path_test = path_dataset3+"Test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = ResNet_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: 7 has samples: 1433 
Class: 2 has samples: 2243 
Class: 5 has samples: 1853 
Class: 8 has samples: 1403 
Class: 3 has samples: 1403 
Class: 1 has samples: 2213 
Class: 4 has samples: 1973 
Class: 9 has samples: 1463 

Total number of samples:  13984

Time taken: 0.009002208709716797


In [ ]:
model_savepath = savepath+"resnetmodel_d3.pt"
results_savepath = savepath+"resnetmodel_d3_results.pkb"

trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/350], Loss: 2.2000, Accuracy: 25.00%, Validation Accuracy: 11.59%
Epoch [1/10], Step [20/350], Loss: 2.1074, Accuracy: 18.75%, Validation Accuracy: 10.12%
Epoch [1/10], Step [30/350], Loss: 1.9264, Accuracy: 25.00%, Validation Accuracy: 21.89%
Epoch [1/10], Step [40/350], Loss: 2.0483, Accuracy: 21.88%, Validation Accuracy: 19.89%
Epoch [1/10], Step [50/350], Loss: 1.9164, Accuracy: 25.00%, Validation Accuracy: 21.46%
Epoch [1/10], Step [60/350], Loss: 1.8880, Accuracy: 21.88%, Validation Accuracy: 19.99%
Epoch [1/10], Step [70/350], Loss: 1.8844, Accuracy: 18.75%, Validation Accuracy: 18.31%
Epoch [1/10], Step [80/350], Loss: 2.1308, Accuracy: 21.88%, Validation Accuracy: 15.34%
Epoch [1/10], Step [90/350], Loss: 1.9996, Accuracy: 12.50%, Validation Accuracy: 23.46%
Epoch [1/10], Step [100/350], Loss: 1.9659, Accuracy: 25.00%, Validation Accuracy: 22.46%
Epoch [1/10], Step [110/350], Loss: 1.8453, Accuracy: 18.75%, Validation Accuracy: 20.39%
Epoch [1/10], Step 

In [84]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)

Model Evaluation Results on 4590 test samples
Test Accuracy:  95.77342047930283
Test F1 Score:  tensor(0.9577)
Test Precision & Recall: (tensor(0.9575), tensor(0.9546))
Test AUC score: tensor(0.1934)


Transfer Learning Models:

Alexnet with dataset 1

In [85]:
nclasses=15

In [86]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [87]:
# layers = AlexNet_model.state_dict().keys()

for name, param in AlexNet_model.named_parameters():
    if name.startswith('features'):
        param.requires_grad = False

In [88]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [89]:
for name, param in AlexNet_model.named_parameters():
        print("Layer: {} has requires_grad set to: {}".format(name, param.requires_grad))

Layer: features.0.weight has requires_grad set to: False
Layer: features.0.bias has requires_grad set to: False
Layer: features.3.weight has requires_grad set to: False
Layer: features.3.bias has requires_grad set to: False
Layer: features.6.weight has requires_grad set to: False
Layer: features.6.bias has requires_grad set to: False
Layer: features.8.weight has requires_grad set to: False
Layer: features.8.bias has requires_grad set to: False
Layer: features.10.weight has requires_grad set to: False
Layer: features.10.bias has requires_grad set to: False
Layer: classifier.1.weight has requires_grad set to: True
Layer: classifier.1.bias has requires_grad set to: True
Layer: classifier.4.weight has requires_grad set to: True
Layer: classifier.4.bias has requires_grad set to: True
Layer: classifier.6.weight has requires_grad set to: True
Layer: classifier.6.bias has requires_grad set to: True


In [90]:
AlexNet_model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [91]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [92]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = AlexNet_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: SPEED_LIMIT_80 has samples: 192 
Class: SPEED_LIMIT_60 has samples: 190 
Class: NO_STOPPING_OR_STANDING has samples: 242 
Class: COMPULSARY_KEEP_RIGHT has samples: 223 
Class: GAP_IN_MEDIAN has samples: 180 
Class: CROSS_ROAD has samples: 140 
Class: PEDESTRIAN_CROSSING has samples: 121 
Class: SPEED_LIMIT_50 has samples: 200 
Class: HORN_PROHIBITED has samples: 160 
Class: SPEED_LIMIT_40 has samples: 170 
Class: NO_ENTRY has samples: 174 
Class: HUMP_OR_ROUGH_ROAD has samples: 101 
Class: SPEED_LIMIT_30 has samples: 238 
Class: LEFT_TURN_PROHIBITED has samples: 126 
Class: SPEED_LIMIT_70 has samples: 160 

Total number of samples:  2617

Time taken: 0.00964665412902832


In [ ]:
model_savepath = savepath+"alexnetmodel_tl_d1.pt"
results_savepath = savepath+"alexnetmodel_tl_d1_results.pkb"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/66], Loss: 1.8880, Accuracy: 37.50%, Validation Accuracy: 43.98%
Epoch [1/10], Step [20/66], Loss: 2.2928, Accuracy: 46.88%, Validation Accuracy: 56.02%
Epoch [1/10], Step [30/66], Loss: 1.1296, Accuracy: 50.00%, Validation Accuracy: 62.52%
Epoch [1/10], Step [40/66], Loss: 1.1000, Accuracy: 56.25%, Validation Accuracy: 59.08%
Epoch [1/10], Step [50/66], Loss: 0.7677, Accuracy: 78.12%, Validation Accuracy: 64.63%
Epoch [1/10], Step [60/66], Loss: 1.1343, Accuracy: 53.12%, Validation Accuracy: 70.36%
Epoch [2/10], Step [10/66], Loss: 0.9170, Accuracy: 68.75%, Validation Accuracy: 64.05%
Epoch [2/10], Step [20/66], Loss: 0.6329, Accuracy: 75.00%, Validation Accuracy: 67.69%
Epoch [2/10], Step [30/66], Loss: 0.8261, Accuracy: 68.75%, Validation Accuracy: 62.72%
Epoch [2/10], Step [40/66], Loss: 0.9550, Accuracy: 68.75%, Validation Accuracy: 69.60%
Epoch [2/10], Step [50/66], Loss: 0.9580, Accuracy: 62.50%, Validation Accuracy: 72.85%
Epoch [2/10], Step [60/66], Loss

In [94]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Model Evaluation Results on 710 test samples
Test Accuracy:  75.77464788732394
Test F1 Score:  tensor(0.7577)
Test Precision & Recall: (tensor(0.8537), tensor(0.7874))
Test AUC score: tensor(0.0510)


ResNet-18 with dataset 1

nclasses=15

In [95]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [96]:
# layers = ResNet_model.state_dict().keys()

for name, param in ResNet_model.named_parameters():
    if not name.startswith('fc'):
        param.requires_grad = False

In [97]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [98]:
for name, param in ResNet_model.named_parameters():
        print("Layer: {} has requires_grad set to: {}".format(name, param.requires_grad))

Layer: conv1.weight has requires_grad set to: False
Layer: bn1.weight has requires_grad set to: False
Layer: bn1.bias has requires_grad set to: False
Layer: layer1.0.conv1.weight has requires_grad set to: False
Layer: layer1.0.bn1.weight has requires_grad set to: False
Layer: layer1.0.bn1.bias has requires_grad set to: False
Layer: layer1.0.conv2.weight has requires_grad set to: False
Layer: layer1.0.bn2.weight has requires_grad set to: False
Layer: layer1.0.bn2.bias has requires_grad set to: False
Layer: layer1.1.conv1.weight has requires_grad set to: False
Layer: layer1.1.bn1.weight has requires_grad set to: False
Layer: layer1.1.bn1.bias has requires_grad set to: False
Layer: layer1.1.conv2.weight has requires_grad set to: False
Layer: layer1.1.bn2.weight has requires_grad set to: False
Layer: layer1.1.bn2.bias has requires_grad set to: False
Layer: layer2.0.conv1.weight has requires_grad set to: False
Layer: layer2.0.bn1.weight has requires_grad set to: False
Layer: layer2.0.bn1.bi

In [99]:
ResNet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [100]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 32
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
x = ResNet_model.to(device)

Device: cuda:0


In [ ]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: SPEED_LIMIT_80 has samples: 192 
Class: SPEED_LIMIT_60 has samples: 190 
Class: NO_STOPPING_OR_STANDING has samples: 242 
Class: COMPULSARY_KEEP_RIGHT has samples: 223 
Class: GAP_IN_MEDIAN has samples: 180 
Class: CROSS_ROAD has samples: 140 
Class: PEDESTRIAN_CROSSING has samples: 121 
Class: SPEED_LIMIT_50 has samples: 200 
Class: HORN_PROHIBITED has samples: 160 
Class: SPEED_LIMIT_40 has samples: 170 
Class: NO_ENTRY has samples: 174 
Class: HUMP_OR_ROUGH_ROAD has samples: 101 
Class: SPEED_LIMIT_30 has samples: 238 
Class: LEFT_TURN_PROHIBITED has samples: 126 
Class: SPEED_LIMIT_70 has samples: 160 

Total number of samples:  2617

Time taken: 0.010135173797607422


In [ ]:
model_savepath = savepath+"resnetmodel_tl_d1.pt"
results_savepath = savepath+"resnetmodel_tl_d1_results.pkb"

trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, model_savepath, results_savepath, device)

Epoch [1/10], Step [10/66], Loss: 2.4429, Accuracy: 18.75%, Validation Accuracy: 21.61%
Epoch [1/10], Step [20/66], Loss: 2.2283, Accuracy: 31.25%, Validation Accuracy: 34.99%
Epoch [1/10], Step [30/66], Loss: 2.1600, Accuracy: 34.38%, Validation Accuracy: 47.04%
Epoch [1/10], Step [40/66], Loss: 1.8542, Accuracy: 46.88%, Validation Accuracy: 50.29%
Epoch [1/10], Step [50/66], Loss: 1.7472, Accuracy: 46.88%, Validation Accuracy: 59.27%
Epoch [1/10], Step [60/66], Loss: 1.5077, Accuracy: 56.25%, Validation Accuracy: 54.88%
Epoch [2/10], Step [10/66], Loss: 1.5007, Accuracy: 56.25%, Validation Accuracy: 65.97%
Epoch [2/10], Step [20/66], Loss: 1.4293, Accuracy: 43.75%, Validation Accuracy: 62.52%
Epoch [2/10], Step [30/66], Loss: 1.4063, Accuracy: 62.50%, Validation Accuracy: 68.64%
Epoch [2/10], Step [40/66], Loss: 0.9628, Accuracy: 68.75%, Validation Accuracy: 69.98%
Epoch [2/10], Step [50/66], Loss: 1.2045, Accuracy: 78.12%, Validation Accuracy: 68.26%
Epoch [2/10], Step [60/66], Loss

In [102]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)

Model Evaluation Results on 710 test samples
Test Accuracy:  70.0
Test F1 Score:  tensor(0.7000)
Test Precision & Recall: (tensor(0.7386), tensor(0.7114))
Test AUC score: tensor(0.1915)
